# Agents Demo

Implementation of a simple agent (a dog) that explores an environment (a garden) in search of food and water.

In [1]:
from agents import Entity
from agents import Agent
from agents import Environment

First we set up the environment, where agents get their perceptions from and execute their actions into. For this we create a class that extends **`agents.Environment`** and implement abstract methods such as `.give_perception()` and `.execute_action()`.

In [2]:
class ParkEnvironment(Environment):
    """An environment where dogs can walk around freely.
    """

    def give_perception(self, agent):
        """See: agents.Environment.give_perception()
        """
        # Return a list of all entities at the agent's location.
        entities = self.list_entities_at(agent.location)
        print(entities)
        return entities

    def execute_action(self, agent, action):
        """See: agents.Environment.execute_action()
        """
        # In this environment an agent can move, eat and drink.
        # When something is eaten or drank, it is removed from the environment.
        if action == 'move':
            agent.move()
        elif action == 'eat':
            food = self.list_entities_at(agent.location, eclass=Food)
            if len(food) > 0:
                if agent.eat(food[0]):
                    self.delete_entity(food[0])
        elif action == 'drink':
            water = self.list_entities_at(agent.location, eclass=Water)
            if len(water) > 0:
                if agent.drink(water[0]):
                    self.delete_entity(water[0])

    def exogenous_change(self):
        """See: agents.Environment.exogenous_change()
        """
        # Nothing needs to be done here.
        pass

    def is_done(self):
        """See: agents.Environment.is_done()
        """
        # By default, an environment is done when no agent is alive to interact with it.
        # But we don't want to kill our dog, so the implementation is changed so that
        # the garden is done when there is no more food or water in it.
        no_edibles = not any(isinstance(entity, Food) or isinstance(entity, Water)
                             for entity in self.entities)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return no_edibles or dead_agents

Now we create the entities the agent can interact with. They extend the **`agents.Entity`** class and need no particular implementation.

In [3]:
class Food(Entity):
    """Dog food from a top brand.
    """
    pass


class Water(Entity):
    """Clean, fresh water.
    """
    pass

And now we create the agent and specify the actions it can perform in the environment. For this we create a class that extends **`agents.Agent`**.

In [4]:
class DogAgent(Agent):
    """A healthy dog that can eat and drink as much as it wants to.
    """

    def move(self):
        self.location += 1
        print('{}: moved to location {}'.format(self, self.location))

    def eat(self, entity):
        if isinstance(entity, Food):
            print('{}: ate food at location {}'.format(self, self.location))
            return True
        return False

    def drink(self, entity):
        if isinstance(entity, Water):
            print('{}: drank water at location {}'.format(self, self.location))
            return True
        return False

Lastly, we specify the agent program, which receives perceptions and returns an action.

In [5]:
def program(perceptions):
    """Returns a string that represents the action to perform, based on the perceptions
    acquired from the environment.
    """
    for p in perceptions:
        if isinstance(p, Food):
            return 'eat'
        if isinstance(p, Water):
            return 'drink'
    return 'move'

Finally, we instantiate all the entities and run the agent program in the environment.

In [6]:
dog = DogAgent(program)
dog.is_alive()

True

In [7]:
food = Food()
water = Water()

In [8]:
park = ParkEnvironment()
park.add_entity(dog, 0)
park.add_entity(food, 3)
park.add_entity(water, 7)

In [9]:
park.run(steps=10)

[<DogAgent>]
<DogAgent>: moved to location 1
[<DogAgent>]
<DogAgent>: moved to location 2
[<DogAgent>]
<DogAgent>: moved to location 3
[<DogAgent>, <Food>]
<DogAgent>: ate food at location 3
[<DogAgent>]
<DogAgent>: moved to location 4
[<DogAgent>]
<DogAgent>: moved to location 5
[<DogAgent>]
<DogAgent>: moved to location 6
[<DogAgent>]
<DogAgent>: moved to location 7
[<DogAgent>, <Water>]
<DogAgent>: drank water at location 7


In [10]:
dog.is_alive()

True